# Comparison of cost vs quality for 50 random cities
### Eric P. Hanson

We will use `TravelingSalesmanExact` to compute the exact cost and compare to the estimated best costs found by `TravelingSalesmanHeuristics` with various settings of `quality`.
First we load the packages:

In [ ]:
using TravelingSalesmanExact, GLPK
using TravelingSalesmanHeuristics
using TravelingSalesmanExact: ATT, euclidean_distance
using TravelingSalesmanBenchmarks
using Plots
using Printf, Random
gr(fmt=:svg)

For plotting, we will override `TravelingSalesmanExact`'s `plot_cities` function to use `Plots`, since we load that package anyway. We also add a mutating version.

In [ ]:
function TravelingSalesmanExact.plot_cities(cities; kwargs...)
    n = length(cities)
    inc(a) = a == n ? one(a) : a + 1
    Plots.plot([cities[inc(j)][1] for j = 0:n], [cities[inc(j)][2] for j = 0:n]; kwargs...)
    Plots.plot!([cities[inc(j)][1] for j = 0:n], [cities[inc(j)][2] for j = 0:n]; st=:scatter, label="City locations")
end

Then we generate a cost matrix by choosing cities at random, although with a fixed seed for reproducibility:

In [ ]:
Random.seed!(155);
N = 50
cities = [100*rand(2) for _ = 1:N]
cost = [ euclidean_distance(cities[i], cities[j]) for i=1:N, j=1:N ]

Now we will compute an optimal tour and cost, and plot these versus those found by the `tsp_solve` function of `TravelingSalesmanHeuristics`.

In [ ]:
t_exact, c_exact = get_optimal_tour(cost, with_optimizer(GLPK.Optimizer))

c(q) = solve_tsp(cost; quality_factor = q)[2]

qs = range(10, stop = 100, step = 10)

plot(qs, c, xlabel="quality", ylabel="Cost", label="solve_tsp 1", title="random cities")
for j = 2:5
    plot!(qs, c, label="solve_tsp $j")
end
hline!([c_exact], label="Exact cost")

We've run `tsp_solve` five times for each quality, since the cost will vary from run to run due to the randomness of the heuristics.

We can compare plots of the tours; first, an optimal tour:

In [ ]:
plot_cities(cities[t_exact], title="Optimal tour", label=@sprintf("cost=%.2f", c_exact))

A heuristically-derived tour with `quality_factor` of `100`:

In [ ]:
t_heuristic, c_heuristic = solve_tsp(cost; quality_factor = 100)
plot_cities(cities[t_heuristic], title="Heuristic tour, `quality_factor` = 100", label=@sprintf("cost=%.2f", c_heuristic))

A heuristically-derived tour with `quality_factor` of `40`:

In [ ]:
t_heuristic, c_heuristic = solve_tsp(cost; quality_factor = 40)
plot_cities(cities[t_heuristic], title="Heuristic tour, `quality_factor` = 40", label=@sprintf("cost=%.2f", c_heuristic))

We can repeat this several times to compare various choices of random cities.

In [ ]:
function compare_cities(N)
    cities = [100*rand(2) for _ = 1:N]
    cost = [ euclidean_distance(cities[i], cities[j]) for i=1:N, j=1:N ]
    t_exact, c_exact = get_optimal_tour(cost, with_optimizer(GLPK.Optimizer))

    c(q) = solve_tsp(cost; quality_factor = q)[2]

    qs = range(10, stop = 100, step = 10)

    line_plt = plot(qs, c, xlabel="quality", ylabel="Cost", label="solve_tsp 1", title="random cities")
    for j = 2:5
        plot!(qs, c, label="solve_tsp $j")
    end
    hline!([c_exact], label="Exact cost")

    opt_city_plt = plot_cities(cities[t_exact], title="Optimal tour", label=@sprintf("cost=%.2f", c_exact))

    t_heuristic, c_heuristic = solve_tsp(cost; quality_factor = 100)
    heuristic_city_plt_1 = plot_cities(cities[t_heuristic], title="Heuristic tour, `quality_factor` = 100", label=@sprintf("cost=%.2f", c_heuristic))

    t_heuristic_2, c_heuristic_2 = solve_tsp(cost; quality_factor = 40)
    heuristic_city_plt_2 = plot_cities(cities[t_heuristic_2], title="Heuristic tour, `quality_factor` = 40", label=@sprintf("cost=%.2f", c_heuristic_2))
    return line_plt, opt_city_plt, heuristic_city_plt_1, heuristic_city_plt_2
end

for j = 1:5
    plts = compare_cities(50)
    for plt in plts
        display(plt)
    end
end

In [ ]:
TravelingSalesmanBenchmarks.bench_footer(WEAVE_ARGS[:file])